In [1]:
%pylab inline
import time
import numpy as np
from segbench import csaAssign, correspondPixels, matchEdgeMaps2D, matchEdgeMaps3D

Populating the interactive namespace from numpy and matplotlib


In [2]:
data = np.array([[1, 4, 3],
    [2, 5, 3],
    [3, 6, 3],
    [1, 5, 1],
    [2, 6, 1],
    [3, 4, 5]])
n = 6
e1 = csaAssign(n,data)
print e1

[[ 1.  5.  1.]
 [ 2.  6.  1.]
 [ 3.  4.  5.]]


In [3]:
n = 1000
dg = np.random.rand(n,n)>.5
m = np.sum(dg.flat)
i = np.array(dg.ravel().nonzero()).astype(float)
g1 = 1+np.floor(np.divide(i,n))
g2 = (1+np.floor(np.divide(i,n)),
     1+ np.mod(i,n)+n,
     1+np.floor(np.random.rand(1,m))*1000)
g2 = np.array(np.vstack(g2)).T
t = time.time()
e2 = csaAssign(n*2,g2)
print '{:.3e}s'.format(time.time() - t)
esum = np.sum(e2,axis=0)
if not all(esum[0:2]==(n*(n+1)/2,n*(n+1)/2+n*n)): raise ValueError("bug")
if not np.sum(esum[0:2])==(2*n*(2*n+1)/2): raise ValueError("bug")
print n,m, e2[3][:]

9.223e-02s
1000 499843 [  4.00000000e+00   2.00000000e+03   1.00000000e+00]


In [4]:
for n in np.round(np.logspace(0,2,10)).astype(int):
    for thresh in np.linspace(0,1,10):
        print 'n={:d} thresh={:1.2e}'.format(n,thresh),
        bmap1 = (np.random.rand(n,n)>thresh).astype(float)
        bmap2 = (np.random.rand(n,n)>thresh).astype(float)

        match1 = zeros_like(bmap1)
        match2 = zeros_like(bmap2)
        t = time.time()
        cost,oc,match1,match2 = correspondPixels(bmap1,bmap2,.005)
        print '{:1.2e}s'.format(time.time() - t),
        npix = len(bmap1.nonzero()[0]) + len(bmap2.nonzero()[0])
        ncor = len(match1.nonzero()[0]) + len(match1.nonzero()[0])
        nout = len(((bmap1>0)&(match1==0)).nonzero()[0]) + len(((bmap2>0)&(match2==0)).nonzero()[0])
        print npix,ncor,nout
        if ncor+nout!=npix: raise ValueError("bug")
        match1f = match1.ravel()
        idx1 = match1f.astype(int)
        a1 = match1f.nonzero()[0]
        a2 = match1f[idx1.nonzero()[0]].ravel()
        a = vstack((a1,a2)).T


        match2f = match2.ravel()
        idx2 = match2f.astype(int)
        b1 = match2f.nonzero()[0]
        b2 = match2f[idx2.nonzero()[0]].ravel()
        b = vstack((b2,b1)).T
        b = b[b[:,0].argsort(),]
        if sum(a-b): raise ValueError("bug")

n=1 thresh=0.00e+00 2.22e-04s 2 2 0
n=1 thresh=1.11e-01 9.11e-05s 2 2 0
n=1 thresh=2.22e-01 2.91e-05s 1 0 1
n=1 thresh=3.33e-01 4.79e-05s 2 2 0
n=1 thresh=4.44e-01 3.50e-05s 1 0 1
n=1 thresh=5.56e-01 1.81e-05s 0 0 0
n=1 thresh=6.67e-01 1.69e-05s 0 0 0
n=1 thresh=7.78e-01 1.62e-05s 0 0 0
n=1 thresh=8.89e-01 1.81e-05s 0 0 0
n=1 thresh=1.00e+00 1.62e-05s 0 0 0
n=2 thresh=0.00e+00 8.58e-05s 8 8 0
n=2 thresh=1.11e-01 4.60e-05s 6 4 2
n=2 thresh=2.22e-01 5.51e-05s 7 6 1
n=2 thresh=3.33e-01 4.51e-05s 6 4 2
n=2 thresh=4.44e-01 4.01e-05s 6 4 2
n=2 thresh=5.56e-01 1.91e-05s 3 0 3
n=2 thresh=6.67e-01 1.79e-05s 3 0 3
n=2 thresh=7.78e-01 1.69e-05s 1 0 1
n=2 thresh=8.89e-01 1.60e-05s 0 0 0
n=2 thresh=1.00e+00 1.60e-05s 0 0 0
n=3 thresh=0.00e+00 2.53e-04s 18 18 0
n=3 thresh=1.11e-01 2.59e-04s 17 16 1
n=3 thresh=2.22e-01 1.10e-04s 15 12 3
n=3 thresh=3.33e-01 7.51e-05s 13 8 5
n=3 thresh=4.44e-01 1.88e-05s 7 0 7
n=3 thresh=5.56e-01 1.81e-05s 8 0 8
n=3 thresh=6.67e-01 4.39e-05s 6 2 4
n=3 thresh=7.78e-01 5

In [5]:
for n in np.round(np.logspace(0,2,10)).astype(int):
    for thresh in np.linspace(0,1,2):
        print 'n={:d} thresh={:1.2e}'.format(n,thresh),
        bmap1 = (np.random.rand(n,n)>thresh).astype(float)
        bmap2 = (np.random.rand(n,n)>thresh).astype(float)
        t = time.time()
        cost,match1,match2 = matchEdgeMaps2D(bmap1,bmap2,.005,100)
        print '{:1.2e}s'.format(time.time() - t),
        npix = len(bmap1.nonzero()[0]) + len(bmap2.nonzero()[0])
        ncor = len(match1.nonzero()[0]) + len(match1.nonzero()[0])
        nout = len(((bmap1>0)&(match1==0)).nonzero()[0]) + len(((bmap2>0)&(match2==0)).nonzero()[0])
        print ncor,'+',nout,'=',ncor+nout,'==',npix
        if ncor+nout!=npix: raise ValueError("bug")
        match1f = match1.ravel()
        idx1 = match1f.astype(int)
        a1 = match1f.nonzero()[0]
        a2 = match1f[idx1.nonzero()[0]].ravel()
        a = vstack((a1,a2)).T


        match2f = match2.ravel()
        idx2 = match2f.astype(int)
        b1 = match2f.nonzero()[0]
        b2 = match2f[idx2.nonzero()[0]].ravel()
        b = vstack((b2,b1)).T
        b = b[b[:,0].argsort(),]
        if sum(a-b): raise ValueError("bug")

n=1 thresh=0.00e+00 7.30e-05s 2 + 0 = 2 == 2
n=1 thresh=1.00e+00 2.31e-05s 0 + 0 = 0 == 0
n=2 thresh=0.00e+00 8.70e-05s 8 + 0 = 8 == 8
n=2 thresh=1.00e+00 2.29e-05s 0 + 0 = 0 == 0
n=3 thresh=0.00e+00 1.94e-04s 18 + 0 = 18 == 18
n=3 thresh=1.00e+00 2.29e-05s 0 + 0 = 0 == 0
n=5 thresh=0.00e+00 6.42e-04s 50 + 0 = 50 == 50
n=5 thresh=1.00e+00 2.91e-05s 0 + 0 = 0 == 0
n=8 thresh=0.00e+00 1.60e-03s 128 + 0 = 128 == 128
n=8 thresh=1.00e+00 3.10e-05s 0 + 0 = 0 == 0
n=13 thresh=0.00e+00 5.57e-03s 338 + 0 = 338 == 338
n=13 thresh=1.00e+00 8.49e-05s 0 + 0 = 0 == 0
n=22 thresh=0.00e+00 1.85e-02s 966 + 2 = 968 == 968
n=22 thresh=1.00e+00 6.99e-05s 0 + 0 = 0 == 0
n=36 thresh=0.00e+00 5.59e-02s 2582 + 10 = 2592 == 2592
n=36 thresh=1.00e+00 1.35e-04s 0 + 0 = 0 == 0
n=60 thresh=0.00e+00 1.99e-01s 7180 + 20 = 7200 == 7200
n=60 thresh=1.00e+00 3.33e-04s 0 + 0 = 0 == 0
n=100 thresh=0.00e+00 6.26e-01s 19948 + 52 = 20000 == 20000
n=100 thresh=1.00e+00 8.81e-04s 0 + 0 = 0 == 0


In [6]:
%pylab inline
import time
import numpy as np
from segbench import csaAssign, correspondPixels, matchEdgeMaps2D, matchEdgeMaps3D2
for n in np.round(np.logspace(0,6,4,base=2)).astype(int):
    for thresh in np.linspace(.1,.8,4):
        print 'n={:d} thresh={:1.2e}'.format(n,thresh),
        bmap1 = (np.random.rand(n,n,n)>thresh).astype(float)
        bmap2 = (np.random.rand(n,n,n)>thresh).astype(float)

        t = time.time()
        cost,match1,match2 = matchEdgeMaps3D2(bmap1,bmap2,.000005,100)
        print '{:1.4f}s'.format(time.time() - t),
        npix = len(bmap1.nonzero()[0]) + len(bmap2.nonzero()[0])
        ncor = len(match1.nonzero()[0]) + len(match1.nonzero()[0])
        nout = len(((bmap1>0)&(match1==0)).nonzero()[0]) + len(((bmap2>0)&(match2==0)).nonzero()[0])
        print ncor,'+',nout,'=',ncor+nout,'==',npix
        if ncor+nout!=npix: raise ValueError("bug")
        match1f = match1.ravel()
        idx1 = match1f.astype(int)
        a1 = match1f.nonzero()[0]
        a2 = match1f[idx1.nonzero()[0]].ravel()
        a = vstack((a1,a2)).T


        match2f = match2.ravel()
        idx2 = match2f.astype(int)
        b1 = match2f.nonzero()[0]
        b2 = match2f[idx2.nonzero()[0]].ravel()
        b = vstack((b2,b1)).T
        b = b[b[:,0].argsort(),]
        if sum(a-b): raise ValueError("bug")

Populating the interactive namespace from numpy and matplotlib


ImportError: cannot import name matchEdgeMaps3D2